In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
# Version Information
# tensorflow 2.5.0 , Cudnn11.4 and Cuda 11.4.0 , python 3.8

This command shows list of physical devices available for tensorflow. You can see GPU listed here. If you have NVIDIA GPU you need to install CUDA toolkit and cuDNN as per instruction on this webpage. Without proper installation you will not see GPU in list of devices

https://shawnhymel.com/1961/how-to-install-tensorflow-with-gpu-support-on-windows/

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 93s 1us/step


In [5]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [6]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [7]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 8s 3ms/step - loss: 1.8116 - accuracy: 0.3541
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.6194 - accuracy: 0.4283
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.5403 - accuracy: 0.4574
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4780 - accuracy: 0.4788
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.4293 - accuracy: 0.4963
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3848 - accuracy: 0.5139
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3484 - accuracy: 0.5273
Epoch 8/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.3145 - accuracy: 0.5390
Epoch 9/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.2838 - accuracy: 0.5489
Epoch 10/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.250

GPU TIME 1min and 3s for 10 epochs

In [9]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.8155 - accuracy: 0.3518
Epoch 2/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.6266 - accuracy: 0.4255
Epoch 3/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.5436 - accuracy: 0.4574
Epoch 4/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.4839 - accuracy: 0.4762
Epoch 5/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.4340 - accuracy: 0.4961
Epoch 6/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3907 - accuracy: 0.5115
Epoch 7/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3531 - accuracy: 0.5229
Epoch 8/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3187 - accuracy: 0.5357
Epoch 9/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.2856 - accuracy: 0.5470
Epoch 10/10
1563/1563 [==============================] - 47s 30m

CPU TIME 7min and 48s for 10 epochs.

You can see that GPU(my GTX 1660 SUPER) is almost 7 times faster. We ran only 10 epochs for benchmarking but for actual training we have to run many epochs and also when data volume is big running deep learning without GPU can consume so much time. This is the reason why GPUs are becoming popular in the field of deep learning